Tarea 2 - Inteligencia Artificial

Integrantes:
- Matias Gonzalez
- Matias Ramirez

In [12]:
import pandas as pd
import random
import itertools

valores_mano = {
    "Carta Alta": 0,
    "Un Par": 1,
    "Dos Pares": 2,
    "Trío": 3,
    "Escalera": 4,
    "Color": 5,
    "Full House": 6,
    "Poker": 7,
    "Escalera de Color": 8,
    "Escalera Real": 9
}

# ---------- Definición de la Mesa ----------
class Mesa:
    def __init__(self):
        self.cartas = self.generar_baraja()
        self.comunes = []
        self.bote = 0
        self.apuesta_actual = 0

    @staticmethod
    def generar_baraja():
        valores = ["2", "3", "4", "5", "6", "7", "8", "9", "T", "J", "Q", "K", "A"]
        palos = ["h", "d", "c", "s"]
        return [f"{valor}{palo}" for valor in valores for palo in palos]

    def repartir_mano(self):
        return [
            self.cartas.pop(random.randint(0, len(self.cartas) - 1)) for _ in range(2)
        ]

    def repartir_comunes(self, num):
        self.comunes.extend(
            [
                self.cartas.pop(random.randint(0, len(self.cartas) - 1))
                for _ in range(num)
            ]
        )


# ---------- Definición del Jugador ----------
class Jugador:
    def __init__(self, nombre, fichas=1000, es_bot=False):
        self.nombre = nombre
        self.fichas = fichas
        self.mano = None
        self.es_bot = es_bot

    def recibir_mano(self, mano):
        """Asigna una mano al jugador."""
        self.mano = mano

    def decidir_accion(self, mesa, q, limite):
        if self.es_bot:
            return decidir_accion_bot(self, mesa, q, limite)
        else:
            # Logica de Humano
            print(f'{self.nombre}, tu mano: {self.mano}')
            print(f'Tu saldo actual: {self.fichas}')
            print(f'Pozo actual en la mesa: {mesa.bote}')
            print(f'Las cartas comunes en la mesa: {mesa.comunes}')

            while True:
                accion = input("¿Qué acción deseas tomar? (fold/call/raise): ").strip().lower()

                if accion == "fold":
                    return "fold"
                elif accion == "call":
                    return mesa.apuesta_actual
                elif accion == "raise":
                    while True:
                        try:
                            cantidad_a_aumentar = int(input(f"Ingrese la cantidad para aumentar: "))
                            if cantidad_a_aumentar > self.fichas:
                                print("No tienes suficientes fichas para esta apuesta.")
                            else:
                                return cantidad_a_aumentar
                        except ValueError:
                            print("Por favor, ingrese un número válido.")
                else:
                    print("Acción no válida. Por favor, elige entre fold, call o raise.")



# ---------- Definición de la Ronda ----------
class Ronda:
    def __init__(self, mesa, q, limite):
        self.mesa = mesa
        self.jugadores = []
        self.apuestas = {}
        self.q = q
        self.limite = limite

    def agregar_jugador(self, jugador):
        self.jugadores.append(jugador)
        self.apuestas[jugador] = 0

    def apostar(self, jugador, cantidad):
        self.mesa.bote += cantidad
        self.apuestas[jugador] += cantidad
        self.mesa.apuesta_actual = max(self.mesa.apuesta_actual, cantidad)

    def encontrar_ganador(self):
        print()
        print("Evaluando Ganador....")
        print("---------------------------------------")
        mejor_valor = -1
        ganador = None

        print(f'Mesa: {self.mesa.comunes}')
        for jugador in self.jugadores:
            mano_completa = jugador.mano + self.mesa.comunes
            valor_mano = evaluar_mano(jugador,mano_completa)
            print(f'{jugador.nombre} tiene {valor_mano} : {jugador.mano}')

            if valores_mano[valor_mano] > mejor_valor:
                ganador = jugador
                mejor_valor = valores_mano[valor_mano]

        return ganador

    def jugar(self):
        print("Comienza la ronda")
        print(f'Jugadores: {[jugador.nombre for jugador in self.jugadores]}')
        print(f'Fichas iniciales: {[jugador.fichas for jugador in self.jugadores]}')
        print(f'Fichas en la mesa: {self.mesa.bote}')
        print("-----------------------------------------------")
        print(f'Los jugadores recibirán 2 cartas')

        for jugador in self.jugadores:
            mano_repartido = self.mesa.repartir_mano()
            jugador.recibir_mano(mano_repartido)
            print(f'{jugador.nombre} recibe {mano_repartido}')
        num_cartas_comunes = [0, 3, 1, 1]
        for i in range(4):
            if len(self.jugadores)<2:
              ganador = self.jugadores[0]
              print(f'El ganador es: {ganador.nombre} y lleva {self.mesa.bote}')
              ganador.fichas += self.mesa.bote
              break
            if i == 0:
              print()
              print("Empezamos el Pre-Flop")
              self.mesa.apuesta_actual = self.limite
            elif i == 1:
              print()
              print("Empezamos el Flop")
            elif i==2:
              print()
              print("Empezamos el turn")
            else:
              print("Empezamos el River")
            print(f'Jugadores: {[jugador.nombre for jugador in self.jugadores]}')
            print(f'Fichas actuales: {[jugador.fichas for jugador in self.jugadores]}')
            print(f'Fichas en la mesa: {self.mesa.bote}')
            print("-----------------------------------------------")
            reparto_comunes = num_cartas_comunes[i]
            self.mesa.repartir_comunes(reparto_comunes)
            print(f'Apuesta actual en la mesa: {self.mesa.apuesta_actual}')
            print(f'Se reparte {reparto_comunes} cartas')
            print(self.mesa.comunes)
            print()

            for jugador in self.jugadores:
                print(f'Juega {jugador.nombre}')
                accion = jugador.decidir_accion(self.mesa, self.q, self.limite)

                if accion == "fold":
                    print(f'{jugador.nombre} decide hacer fold')
                    self.jugadores.remove(jugador)
                elif accion == "all-in":
                    print(f'{jugador.nombre} decide hacer all in')
                    self.apostar(jugador, jugador.fichas)
                    jugador.fichas = 0
                else:
                  if accion == self.mesa.apuesta_actual:
                    print(f'{jugador.nombre} decide apostar {accion}')
                    self.apostar(jugador, accion)
                    jugador.fichas -= accion
                  else:
                    print(f'{jugador.nombre} decide aumentar {accion}')
                    self.apostar(jugador, accion)
                    jugador.fichas -= accion

        ganador = self.encontrar_ganador()
        print(f'El ganador es: {ganador.nombre} y lleva {self.mesa.bote}')
        ganador.fichas += self.mesa.bote


# ---------- Funciones auxiliares ----------
def evaluar_mano(jugador, mano):
    valores = "23456789TJQKA"
    palos = "hdcs"

    def es_escalera(valores_cartas):
        return len(set(valores_cartas)) == 5 and max(valores_cartas) - min(valores_cartas) == 4

    def contar_cartas(valores_cartas):
        contador = {}
        for valor in valores_cartas:
            if valor in contador:
                contador[valor] += 1
            else:
                contador[valor] = 1
        return contador

    def determinar_valor_mano(mano):
        valores_cartas = sorted([valores.index(carta[0]) for carta in mano])
        palos_cartas = [carta[1] for carta in mano]
        contador_cartas = contar_cartas(valores_cartas)

        if es_escalera(valores_cartas) and len(set(palos_cartas)) == 1:
            if max(valores_cartas) == valores.index('A') and min(valores_cartas) == valores.index('T'):
                return "Escalera Real"
            else:
                return "Escalera de Color"

        if 4 in contador_cartas.values():
            return "Poker"

        if 3 in contador_cartas.values() and 2 in contador_cartas.values():
            return "Full House"

        if len(set(palos_cartas)) == 1:
            return "Color"

        if es_escalera(valores_cartas):
            return "Escalera"

        if 3 in contador_cartas.values():
            return "Trío"

        pares = list(contador_cartas.values()).count(2)
        if pares == 2:
            return "Dos Pares"
        elif pares == 1:
            return "Un Par"

        return "Carta Alta"

    return determinar_valor_mano(mano)

def obtener_probabilidad_pre_flop(mano):
    mano_str = "".join(sorted([card[0] for card in mano]))
    pattern = f"{mano_str}|{mano_str[::-1]}"
    if mano[0][0] == mano[1][0]:  # Es un par
        matched_rows = df_probabilidades[
            df_probabilidades["Cards"].str.contains(pattern, case=False, regex=True)
        ]
    else:
        if mano[0][1] == mano[1][1]:  # Las cartas son del mismo palo
            mano_str += "s"
            matched_rows = df_probabilidades[
                df_probabilidades["Cards"].str.contains(pattern, case=False, regex=True)
            ]
        else:
            mano_str += "o"
            matched_rows = df_probabilidades[
                df_probabilidades["Cards"].str.contains(pattern, case=False, regex=True)
            ]

    if matched_rows.empty:
        print(f"No se encontró la mano: {mano_str} en el DataFrame")
        return 0.0

    probabilidad = matched_rows["Rank"].values[0]
    probabilidad /= 169.0
    return probabilidad


def decidir_accion_bot(jugador, mesa, q, limite):
    if len(mesa.comunes) == 0:
        probabilidad = obtener_probabilidad_pre_flop(jugador.mano)
    else:
        probabilidad = obtener_probabilidad_despues_flop(jugador.mano, mesa.comunes)

    if probabilidad > q: # Las probabilidades son buenas, se hace raise
      if mesa.apuesta_actual*1.25 >= jugador.fichas:
        return "all-in"
      else:
        return min(mesa.apuesta_actual*1.25, jugador.fichas)
    elif probabilidad > 1-q: # Las probabilidades son normales, se juega
        return mesa.apuesta_actual
    else: # Probabilidades son malas
        return "fold"

def obtener_probabilidad_despues_flop(mano_jugador, cartas_mesa):
    valores = ["2", "3", "4", "5", "6", "7", "8", "9", "T", "J", "Q", "K", "A"]
    palos = ["h", "d", "c", "s"]
    baraja_total = [f"{valor}{palo}" for valor in valores for palo in palos]

    # Quitamos las cartas que ya se encuentran en la mesa y las que tiene el jugador
    for carta in mano_jugador + cartas_mesa:
        baraja_total.remove(carta)

    # Número de cartas que faltan para completar el board (5 cartas en total)
    cartas_faltantes = 5 - len(cartas_mesa)

    # Generamos todas las combinaciones posibles de cartas que podrían salir
    posibles_cartas_mesa = list(itertools.combinations(baraja_total, cartas_faltantes))

    # Generamos todas las combinaciones de manos que el adversario podría tener
    manos_adversario = list(itertools.combinations(baraja_total, 2))

    casos_favorables = 0
    total_casos = 0

    for posible_mesa in posibles_cartas_mesa:
        for mano_oponente in manos_adversario:
            mesa_completa = list(cartas_mesa) + list(posible_mesa)

            # Evitar que el oponente tenga cartas que ya están en la mesa
            if set(mano_oponente).intersection(mesa_completa):
                continue

            total_casos += 1
            valor_mano_jugador = evaluar_mano(mano_jugador, list(mano_jugador) + mesa_completa)
            valor_mano_oponente = evaluar_mano(mano_oponente, list(mano_oponente) + mesa_completa)


            # Si el jugador tiene una mano mejor que el oponente, se cuenta como un caso favorable
            if valores_mano[valor_mano_jugador] > valores_mano[valor_mano_oponente]:
                casos_favorables += 1

    return casos_favorables / total_casos if total_casos > 0 else 0



# ---------- Juego principal ----------
def jugar_poker():
    mesa = Mesa()
    q = 0.75
    limite = 100
    jugador1 = Jugador("Jugador1", es_bot=True)
    jugador2 = Jugador("Jugador2", es_bot=True)
    ronda = Ronda(mesa, q, limite)
    ronda.agregar_jugador(jugador1)
    ronda.agregar_jugador(jugador2)
    ronda.jugar()
    print(f"Fichas de {jugador1.nombre}: {jugador1.fichas}")
    print(f"Fichas de {jugador2.nombre}: {jugador2.fichas}")



# Carga la matriz de probabilidades
columnas = ["Rank", "Cards", "Type"]
df_probabilidades = pd.read_csv(
    "https://raw.githubusercontent.com/BrenoCPimenta/Poker-preflop-hand-rank-scraping-to-csv/master/preFlop-rank.csv",
    header=None,
    names=columnas,
)

# Ejecuta el juego
jugar_poker()

Comienza la ronda
Jugadores: ['Jugador1', 'Jugador2']
Fichas iniciales: [1000, 1000]
Fichas en la mesa: 0
-----------------------------------------------
Los jugadores recibirán 2 cartas
Jugador1 recibe ['3c', 'Th']
Jugador2 recibe ['7s', '9s']

Empezamos el Pre-Flop
Jugadores: ['Jugador1', 'Jugador2']
Fichas actuales: [1000, 1000]
Fichas en la mesa: 0
-----------------------------------------------
Apuesta actual en la mesa: 100
Se reparte 0 cartas
[]

Juega Jugador1
Jugador1 decide apostar 100
Juega Jugador2
Jugador2 decide apostar 100

Empezamos el Flop
Jugadores: ['Jugador1', 'Jugador2']
Fichas actuales: [900, 900]
Fichas en la mesa: 200
-----------------------------------------------
Apuesta actual en la mesa: 100
Se reparte 3 cartas
['Kd', '7h', '3d']

Juega Jugador1
Jugador1 decide apostar 100
Juega Jugador2
Jugador2 decide apostar 100

Empezamos el turn
Jugadores: ['Jugador1', 'Jugador2']
Fichas actuales: [800, 800]
Fichas en la mesa: 400
---------------------------------------